# Explore here

In [2]:
# Your code here
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv")
data_uni = data.drop_duplicates()
data_uni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


In [21]:
data.to_csv("../data/raw/data.csv",index=False)

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

col_obj = data_uni.select_dtypes(include='object').columns.tolist()
col_num = data_uni.select_dtypes(exclude='object').columns.tolist()
def eliminar_atipicos(datos, columnas):
    new_data = datos
    for i in columnas:
        q1=new_data[i].quantile(0.25)
        q3=new_data[i].quantile(0.75)
        iqr = q3-q1
        low_lim = q1 - 1.5*iqr
        hi_lim = q3 + 1.5*iqr
        rem = new_data[(new_data[i]>=hi_lim) | (new_data[i]< low_lim)]
        new_data = new_data.drop(index=rem.index)
    return new_data.copy()
data_num_clean = eliminar_atipicos(data_uni,col_num)
# fig, axis = plt.subplots(figsize = (106,106))
# sns.heatmap(data_num_clean[col_num].corr(), annot = True, fmt = ".2f")

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
norm_features = scaler.fit_transform(data_uni[col_num])
data_scal = pd.DataFrame(norm_features, index = data_uni.index, columns = col_num)
data_scal["Heart disease_number"] = data_uni["Heart disease_number"]

correlation_matrix = data_scal.corr(numeric_only=True)
cor_target = correlation_matrix['Heart disease_number'].abs().sort_values(ascending=False)
top_features = cor_target[cor_target > 0.5]  # puedes cambiar el umbral
print(top_features[:20])
data_top = data_scal[top_features[:20].index]
X = data_top.drop(columns=["Heart disease_number"])
Y = data_scal["Heart disease_number"]

Heart disease_number                          1.000000
COPD_number                                   0.997023
CKD_number                                    0.995200
anycondition_number                           0.993327
60-69                                         0.991934
High school diploma only 2014-18              0.991331
Population Aged 60+                           0.991230
70-79                                         0.990426
diabetes_number                               0.989505
Obesity_number                                0.989187
county_pop2018_18 and older                   0.987918
TOT_POP                                       0.987511
POP_ESTIMATE_2018                             0.987511
50-59                                         0.987303
Total Population                              0.986991
Some college or associate's degree 2014-18    0.985867
Civilian_labor_force_2018                     0.981817
Employed_2018                                 0.981331
80+       

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.metrics import mean_squared_error, r2_score

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=8)

model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
r2_score(y_test,y_pred)

/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8126531760371917

In [22]:
x_train.to_csv("../data/processed/x_train.csv", index=False)
x_test.to_csv("../data/processed/x_test.csv", index=False)
y_train.to_csv("../data/processed/y_train.csv", index=False)
y_test.to_csv("../data/processed/y_test.csv", index=False)

In [20]:
from sklearn.linear_model import LogisticRegression, Lasso
las_model = Lasso()
las_model.fit(x_train,y_train)
y_pred_las = las_model.predict(x_test)
r2_score(y_test,y_pred_las)

/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.226e+08, tolerance: 6.299e+07
  model = cd_fast.enet_coordinate_descent(


0.9951294421825511